# Create Tables

In [1]:
# Packages
import pandas as pd
import os
import dataframe_image as dfi # NOTE: YOU MUST HAVE GOOGLE CHROME INSTALLED FOR THIS TO WORK CORRECTLY
import matplotlib.pyplot as plt

## Set Classifier Name

In [2]:
classifier_name = 'Logistic Regression'

## List of Rating Models and Most Complex Model

In [3]:
model_names = ['rating_model_1', 'rating_model_2', 'rating_model_3']
clean_model_names = ["Altman's Z", 'Financial Variables and Sector', 'Financial Variables, Sector, and NLP Features']
# set most_complex_model 
most_complex_model = 'rating_model_3'

## Variable Index

In [4]:
# Load variable index
variable_index = pd.read_excel('../../../../Variable Index.xlsx')
variable_index

,column_name,Clean Column Name,Variable Type,Data Type,Ratio?,Notes,Rating Model 1,Rating Model 2,Rating Model 3,Change Model 1,Change Model 2,Change Model 3
0,Altman_Z,Altman's Z Score,Altman's Z Score,Numeric,Y,NaN,X,NaN,NaN,X,NaN,NaN
1,EBIT,EBIT,Constructed for Altman's Z,Numeric,NaN,NaN,NaN,X,X,NaN,X,X
2,common_plus_preferred_stock,Common Plus Preferred Stock,Constructed for Altman's Z,Numeric,NaN,NaN,NaN,X,X,NaN,X,X
3,workingCapital,Working Capital,Constructed for Altman's Z,Numeric,NaN,NaN,NaN,X,X,NaN,X,X
4,Ratio_A,Ratio A,Constructed for Altman's Z,Numeric,Y,NaN,NaN,X,X,NaN,X,X
...,...,...,...,...,...,...,...,...,...,...,...,...
200,operatingCashFlowPerShare_diff,Difference in Operating Cash Flow Per Share fr...,Additional Change Ratios,Numeric,NaN,"Primarily for changes models, but can be used ...",NaN,NaN,NaN,NaN,X,X
201,freeCashFlowPerShare_diff,Difference in Free Cash Flow Per Share from pr...,Additional Change Ratios,Numeric,NaN,"Primarily for changes models, but can be used ...",NaN,NaN,NaN,NaN,X,X
202,cashPerShare_diff,Difference in Cash Per Share from prior fixed ...,Additional Change Ratios,Numeric,NaN,"Primarily for changes models, but can be used ...",NaN,NaN,NaN,NaN,X,X
203,operatingCashFlowToSales_diff,Difference in Operating Cash Flow to Sales fro...,Additional Change Ratios,Numeric,NaN,"Primarily for changes models, but can be used ...",NaN,NaN,NaN,NaN,X,X


## Model Comparison

In [5]:
def get_model_comparison_row(model_name, clean_model_name):
    '''
    Given the model name and clean model name, this function returns the model comparison row.
    '''

    # Load close_exact_dict
    close_exact_dict = pd.read_pickle('../../../../Output/Modelling/' + classifier_name + '/' + model_name + '/' + model_name + '_close_exact_dict.pkl')
    # Version with each item rounded to 4 decimal places
    close_exact_dict_rounded = {k: round(v, 4) for k, v in close_exact_dict.items()}
    # Unpack
    exact_predictions_share = close_exact_dict_rounded['exact_predictions_share']
    close_predictions_share = close_exact_dict_rounded['close_predictions_share']

    # Load acc_f1_majority
    acc_f1_majority = pd.read_pickle('../../../../Output/Modelling/' + classifier_name + '/' + model_name + '/' + model_name + '_acc_f1_majority.pkl')
    # Version with each item rounded to 2 decimal places
    acc_f1_majority_rounded = {k: round(v, 4) for k, v in acc_f1_majority.items()}
    # Unpack
    accuracy = acc_f1_majority_rounded['accuracy']
    f1 = acc_f1_majority_rounded['f1_score']
    majority_baseline = acc_f1_majority_rounded['majority_baseline']

    # Check exact_predictions_share == accuracy
    print('exact predictions share == accuracy:', exact_predictions_share == accuracy)

    # Get weighted average precision and recall from classification report
    classification_report = pd.read_pickle('../../../../Output/Modelling/' + classifier_name + '/' + model_name + '/' + model_name + '_classification_report.pkl')
    # Convert classification report string to dataframe
    classification_report_lines = classification_report.split('\n')
    # split on spaces within and drop blanks
    classification_report_data = [line.split() for line in classification_report_lines if line]
    # drop lists begining with 'precision', 'accuracy', 'macro', 'weighted'
    classification_report_data = [line for line in classification_report_data if line[0] in ['weighted']]
    # Unpack
    weighted_avg_precision = classification_report_data[0][2]
    weighted_avg_recall = classification_report_data[0][3]

    # Create dataframe row
    model_comparison_row = pd.DataFrame({
        'Model/Baseline': [clean_model_name],
        'Accuracy': [accuracy],
        'Weighted Average Precision': [weighted_avg_precision],
        'Weighted Average Recall': [weighted_avg_recall],
        'F1 Score': [f1],
        'Share 1 Rating Or Less From Actual': [close_predictions_share]
    })

    # Return row
    return model_comparison_row, majority_baseline

## Model Comparison Table - Include and Exclude Previous Rating Versions

In [6]:
# Iterate over include_exclude_previous
for include_exclude_previous in ['exclude_previous_', 'include_previous_']:

    # Create list of df rows
    model_comparison_rows = []
    majority_baselines = []
    for model_name, clean_model_name in zip(model_names, clean_model_names):
        model_comparison_row, majority_baseline = get_model_comparison_row(include_exclude_previous + model_name, clean_model_name)
        model_comparison_rows.append(model_comparison_row)
        majority_baselines.append(majority_baseline)

    # Concatenate rows
    model_comparison_df = pd.concat(model_comparison_rows)

    # Check majority baselines are the same
    print('Majority baselines are the same:', all([majority_baseline == majority_baselines[0] for majority_baseline in majority_baselines]))
    # Add row with Model/Baseline = 'Most Common Class Baseline' and Accuracy = majority_baseline[0]
    model_comparison_df = pd.concat([model_comparison_df, pd.DataFrame({
        'Model/Baseline': ['Most Common Class Baseline'],
        'Accuracy': [majority_baselines[0]],
        'Weighted Average Precision': [''],
        'Weighted Average Recall': [''],
        'F1 Score': [''],
        'Share 1 Rating Or Less From Actual': ['']
    })])

    # Export to Excel
    model_comparison_df.to_excel('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'model_comparison_df.xlsx', index = False)

    if include_exclude_previous == 'exclude_previous_':
        exclude_previous_model_comparison_df = model_comparison_df
    else:
        include_previous_model_comparison_df = model_comparison_df

    # Export to LaTeX
    # Format columns
    for col in model_comparison_df.columns:
        model_comparison_df[col] = model_comparison_df[col].apply(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
    # Rename 'Share 1 Rating Or Less From Actual' to 'Share $le$ Rating From Actual'
    model_comparison_df.rename(columns={'Share 1 Rating Or Less From Actual': 'Share $\\le$ 1 Rating From Actual'}, inplace=True)
    # Center all columns
    lt_string = model_comparison_df.to_latex(index=False, column_format='c' * len(model_comparison_df.columns), escape=False)
    latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'model_comparison_df.tex', 'w') as f:
        f.write(latex_with_font_size)

    # Middle version latex - keep only columns 'Model/Baseline', 'Accuracy', 'Share $\\le$ 1 Rating From Actual'
    model_comparison_df_middle = model_comparison_df[['Model/Baseline', 'Accuracy', 'Share $\\le$ 1 Rating From Actual']]
    # Center all columns
    lt_string = model_comparison_df_middle.to_latex(index=False, column_format='c' * len(model_comparison_df_middle.columns), escape=False)
    latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'model_comparison_df_middle.tex', 'w') as f:
        f.write(latex_with_font_size)

    # Smaller version latex - keep only columns 'Model/Baseline', 'Accuracy'
    model_comparison_df_smaller = model_comparison_df[['Model/Baseline', 'Accuracy']]
    # Center all columns
    lt_string = model_comparison_df_smaller.to_latex(index=False, column_format='c' * len(model_comparison_df_smaller.columns), escape=False)
    latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'model_comparison_df_smaller.tex', 'w') as f:
        f.write(latex_with_font_size)

    print(model_comparison_df)

    # If include_previous, add a row that is the share of items with "Same" in test data
    if include_exclude_previous == 'include_previous_':
        test_data_example = pd.read_excel('../../../../Data/Predictions/Logistic Regression/rating_change_model_1/rating_change_model_1_predictions.xlsx')
        print(test_data_example)
        same_share = test_data_example[test_data_example['Change Direction Since Last Fixed Quarter Date'] == 'Same As Last Fixed Quarter Date'].shape[0] / test_data_example.shape[0]
        # round to 4 decimal places
        same_share = round(same_share, 4)
        print('same share')
        print(same_share)
        # Create dataframe row
        same_share_row = pd.DataFrame({
            'Model/Baseline': ['Previous Rating'],
            'Accuracy': [same_share],
            'Weighted Average Precision': [''],
            'Weighted Average Recall': [''],
            'F1 Score': [''],
        })
        # Concatenate rows
        model_comparison_df_ss = pd.concat([model_comparison_df, same_share_row])
        # Export to Excel
        model_comparison_df_ss.to_excel('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'model_comparison_df_ss.xlsx', index = False)

exact predictions share == accuracy: True
exact predictions share == accuracy: True
exact predictions share == accuracy: True
Majority baselines are the same: True
                                  Model/Baseline Accuracy  \
0                                     Altman's Z   0.1923   
0                 Financial Variables and Sector   0.6225   
0  Financial Variables, Sector, and NLP Features   0.6333   
0                     Most Common Class Baseline   0.3247   

  Weighted Average Precision Weighted Average Recall F1 Score  \
0                     0.3231                  0.1923   0.2266   
0                     0.6295                  0.6225   0.6223   
0                     0.6389                  0.6333   0.6329   
0                                                               

  Share $\le$ 1 Rating From Actual  
0                           0.4633  
0                           0.9186  
0                           0.9267  
0                                   
exact predictions s

## Styled DFI Versions for Presentation

In [7]:
# Styling
#print(exclude_previous_model_comparison_df)
exclude_previous_model_comparison_sty = (exclude_previous_model_comparison_df.reset_index(drop=True)[['Model/Baseline', 'Accuracy']]
                            .style
                            .format(precision=2, thousands=",", decimal=".")
                            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                            .hide()
                            .set_properties(**{'text-align': 'center'}))

dfi.export(exclude_previous_model_comparison_sty, '../../../../Output/Modelling/' + classifier_name + '/Tables/' + 'exclude_previous_' + 'model_comparison_df.png')
#exclude_previous_model_comparison_sty

include_previous_model_comparison_sty = (include_previous_model_comparison_df.reset_index(drop=True)[['Model/Baseline', 'Accuracy']]
                            .style
                            .format(precision=2, thousands=",", decimal=".")
                            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                            .hide()
                            .set_properties(**{'text-align': 'center'}))

dfi.export(include_previous_model_comparison_sty, '../../../../Output/Modelling/' + classifier_name + '/Tables/' + 'include_previous_' + 'model_comparison_df.png')
#include_previous_model_comparison_sty

model_comparison_df_ss = (model_comparison_df_ss.reset_index(drop=True)[['Model/Baseline', 'Accuracy']]
                            .style
                            .format(precision=4, thousands=",", decimal=".")
                            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                            .hide()
                            .set_properties(**{'text-align': 'center'}))

dfi.export(model_comparison_df_ss, '../../../../Output/Modelling/' + classifier_name + '/Tables/' + 'include_previous_' + 'model_comparison_df_ss.png')
#model_comparison_df_ss

## Most Complex Classification Rep

In [8]:
# Iterate over include_exclude_previous
for include_exclude_previous in ['exclude_previous_', 'include_previous_']:

    # Load classificiation report from pickle
    classification_report = pd.read_pickle('../../../../Output/Modelling/' + classifier_name + '/' + include_exclude_previous + most_complex_model + '/' + include_exclude_previous + most_complex_model + '_classification_report.pkl')
    print(classification_report)

    # Convert classification report string to dataframe
    classification_report_lines = classification_report.split('\n')
    # split on spaces within and drop blanks
    classification_report_data = [line.split() for line in classification_report_lines if line]
    # drop lists begining with 'precision', 'accuracy', 'macro', 'weighted'
    classification_report_data = [line for line in classification_report_data if line[0] not in ['precision', 'accuracy', 'macro', 'weighted']]
    # Stack list of rows into dataframe
    classification_report_data = pd.DataFrame(classification_report_data)
    # Set columns to "Rating", "Precision", "Recall", "F1-Score", "Support"
    classification_report_data.columns = ['Rating', 'Precision', 'Recall', 'F1-Score', 'Support']
    # Sort by Rating in correct order: AAA, AA, A, BBB, BB, B, CCC, CC, C, D
    rating_map = {'AAA': 0, 'AA': 1, 'A': 2, 'BBB': 3, 'BB': 4, 'B': 5, 'CCC': 6, 'CC': 7, 'C': 8, 'D': 9}
    classification_report_data['Rating Num'] = classification_report_data['Rating'].map(rating_map)
    classification_report_data = classification_report_data.sort_values(by='Rating Num').drop(columns='Rating Num')
    print(classification_report_data)

    # Export to Excel
    classification_report_data.to_excel('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'Most_Complex_Model_Classification_Report.xlsx', index=False)

    # Export to Latex
    #classification_report_data.to_latex('../../../../Output/Modelling/Logistic Regression/Tables/Most_Complex_Model_Classification_Report.tex', index=False)

    # Export to LaTeX
    # Format columns
    for col in classification_report_data.columns:
        classification_report_data[col] = classification_report_data[col].apply(lambda x: f'{x:.2f}' if isinstance(x, float) else x)
    # Center all columns
    lt_string = classification_report_data.to_latex(index=False, column_format='c' * 5, escape=False)
    latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'Most_Complex_Model_Classification_Report.tex', 'w') as f:
        f.write(latex_with_font_size)

              precision    recall  f1-score   support

           A     0.5874    0.6298    0.6079       208
          AA     0.6316    0.6923    0.6606        52
         AAA     0.8261    0.7917    0.8085        24
           B     0.5864    0.7273    0.6493       154
          BB     0.6275    0.5458    0.5838       284
         BBB     0.6909    0.6281    0.6580       363
           C     1.0000    1.0000    1.0000         4
          CC     0.3333    1.0000    0.5000         2
         CCC     0.5556    0.7692    0.6452        26
           D     1.0000    1.0000    1.0000         1

    accuracy                         0.6333      1118
   macro avg     0.6839    0.7784    0.7113      1118
weighted avg     0.6389    0.6333    0.6329      1118

  Rating Precision  Recall F1-Score Support
2    AAA    0.8261  0.7917   0.8085      24
1     AA    0.6316  0.6923   0.6606      52
0      A    0.5874  0.6298   0.6079     208
5    BBB    0.6909  0.6281   0.6580     363
4     BB    0.6275  0

## Most Complex Hyperparameters

In [9]:
# List to store best parameters dfs
best_params_dfs = []

# Iterate over include_exclude_previous
for include_exclude_previous in ['exclude_previous_', 'include_previous_']:

    # Load pickle
    best_params = pd.read_pickle('../../../../Output/Modelling/' + classifier_name + '/' + include_exclude_previous + most_complex_model + '/' + include_exclude_previous + most_complex_model + '_best_params.pkl')
    print(best_params)

    # Convert to dataframe
    best_params = pd.DataFrame(best_params, index=[0])
    # Set columns to "C", "Class Weighting Strategy", "L1 Ratio", "Multi-Class Strategy", "Penalty", "Solver"
    best_params.columns = ['C', 'Class Weighting Strategy', 'L1 Ratio', 'Multi-Class Strategy', 'Penalty', 'Solver']
    # Replace 'Multi-Class Strategy' values
    best_params['Multi-Class Strategy'] = best_params['Multi-Class Strategy'].replace({'ovr': 'One vs Rest', 'multinomial': 'Multinomial'})
    # Replace 'Penalty' values
    best_params['Penalty'] = best_params['Penalty'].replace({'l1': 'L1', 'l2': 'L2', 'elasticnet': 'Elastic Net', 'none': 'None'})
    # Replace 'Solver' values
    best_params['Solver'] = best_params['Solver'].replace({'newton-cg': 'Newton Conjugate Gradient', 'lbfgs': 'Limited Memory Broyden–Fletcher–Goldfarb–Shanno', 'liblinear': 'Library for Large Linear Classification', 'sag': 'Stochastic Average Gradient', 'saga': 'SAGA'})
    # Replace Class Weighting Strategy values
    best_params['Class Weighting Strategy'] = best_params['Class Weighting Strategy'].replace({'balanced': 'Balanced', None: 'None'})
    
    # Column at the front for whether previous ratings are included or excluded
    best_params.insert(0, 'Previous Ratings', include_exclude_previous[:-1].replace('_', ' ').title())

    # Export smaller version to Excel
    # drop first col
    bp_small = best_params.drop(columns='Previous Ratings')
    bp_small.to_excel('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'Most_Complex_Model_Best_Params.xlsx', index=False)

    # Export to LaTeX
    for col in ['C', 'L1 Ratio']:
        bp_small[col] = bp_small[col].apply(lambda x: '{:,.2f}'.format(x))
    # Center all columns
    lt_string = bp_small.to_latex(index=False, column_format='c' * len(bp_small.columns), escape=False)
    latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'Most_Complex_Models_Best_Params.tex', 'w') as f:
        f.write(latex_with_font_size) 

    # Append to best_params_dfs
    best_params_dfs.append(best_params)

# Concatenate best_params_dfs
best_params = pd.concat(best_params_dfs)
print(best_params)

# Export to Excel
best_params.to_excel('../../../../Output/Modelling/' + classifier_name + '/Tables/Most_Complex_Models_Best_Params.xlsx', index=False)

# Export to Latex
#best_params.to_latex('../../../../Output/Modelling/Logistic Regression/Tables/Most_Complex_Model_Best_Params.tex', index=False)

# Export to LaTeX
for col in ['C', 'L1 Ratio']:
   best_params[col] = best_params[col].apply(lambda x: '{:,.2f}'.format(x))
# Center all columns
lt_string = best_params.to_latex(index=False, column_format='c' * len(best_params.columns), escape=False)
latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
with open('../../../../Output/Modelling/' + classifier_name + '/Tables/Most_Complex_Models_Best_Params.tex', 'w') as f:
    f.write(latex_with_font_size)

{'C': 0.1, 'class_weight': 'balanced', 'l1_ratio': 1.0, 'multi_class': 'ovr', 'penalty': 'elasticnet', 'solver': 'saga'}
{'C': 0.1, 'class_weight': 'balanced', 'l1_ratio': 1.0, 'multi_class': 'ovr', 'penalty': 'elasticnet', 'solver': 'saga'}
   Previous Ratings    C Class Weighting Strategy  L1 Ratio  \
0  Exclude Previous  0.1                 Balanced       1.0   
0  Include Previous  0.1                 Balanced       1.0   

  Multi-Class Strategy      Penalty Solver  
0          One vs Rest  Elastic Net   SAGA  
0          One vs Rest  Elastic Net   SAGA  


## Most Complex Permutation Importance

In [10]:
# Iterate over include_exclude_previous
for include_exclude_previous in ['exclude_previous_', 'include_previous_']:

    # Load data
    permutation_importance = pd.read_parquet('../../../../Output/Modelling/' + classifier_name + '/' + include_exclude_previous + most_complex_model + '/' + include_exclude_previous + most_complex_model + '_permutation_importance.parquet')
    permutation_importance = permutation_importance.sort_values('mean',ascending=False)
    # Set columns to "Feature", "Mean", "Standard Deviation"
    permutation_importance.columns = ['Feature', 'Mean', 'Standard Deviation']
     # Strip 'cat__' and 'num__' from Feature
    permutation_importance['Feature'] = permutation_importance['Feature'].str.replace('cat__', '').str.replace('num__', '')
    # Use variable_index to get feature names
    permutation_importance = permutation_importance.merge(variable_index[['column_name', 'Clean Column Name']], left_on='Feature', right_on='column_name', how='left')
    # Set Clean_Column_Name to Feature if no match
    permutation_importance['Clean Column Name'] = permutation_importance['Clean Column Name'].fillna(permutation_importance['Feature'])
    # Drop Feature and column_name
    permutation_importance = permutation_importance.drop(columns=['Feature', 'column_name'])
    # Clean up names for categorical columns
    previous_rating_mapping = {'rating_on_previous_fixed_quarter_date_AAA': 'Rating on Previous Fixed Quarter Date AAA',
                                'rating_on_previous_fixed_quarter_date_AA': 'Rating on Previous Fixed Quarter Date AA',
                                'rating_on_previous_fixed_quarter_date_A': 'Rating on Previous Fixed Quarter Date A',
                                'rating_on_previous_fixed_quarter_date_BBB': 'Rating on Previous Fixed Quarter Date BBB',
                                'rating_on_previous_fixed_quarter_date_BB': 'Rating on Previous Fixed Quarter Date BB',
                                'rating_on_previous_fixed_quarter_date_B': 'Rating on Previous Fixed Quarter Date B',
                                'rating_on_previous_fixed_quarter_date_CCC': 'Rating on Previous Fixed Quarter Date CCC',
                                'rating_on_previous_fixed_quarter_date_CC': 'Rating on Previous Fixed Quarter Date CC',
                                'rating_on_previous_fixed_quarter_date_C': 'Rating on Previous Fixed Quarter Date C',
                                'rating_on_previous_fixed_quarter_date_D': 'Rating on Previous Fixed Quarter Date D'}
    permutation_importance['Clean Column Name'] = permutation_importance['Clean Column Name'].replace(previous_rating_mapping)
    # Replace 'Sector_' with 'Sector: '
    permutation_importance['Clean Column Name'] = permutation_importance['Clean Column Name'].str.replace('Sector_', 'Sector: ')
    # Rename Clean Column Name to Feature
    permutation_importance = permutation_importance.rename(columns={'Clean Column Name': 'Feature'})
    # Reorder columns to put Feature first
    permutation_importance = permutation_importance[['Feature', 'Mean', 'Standard Deviation']]
    # Rename Mean to 'Mean Accuracy Drop'
    permutation_importance = permutation_importance.rename(columns={'Mean': 'Mean Accuracy Drop'})
    # Rename Feature to 'Permuted Feature'
    permutation_importance = permutation_importance.rename(columns={'Feature': 'Permuted Feature'})
    # Get top 15
    pi_top_15 = permutation_importance.head(15)

    if include_exclude_previous == 'exclude_previous_':
        exclude_previous_pi_top_15 = pi_top_15
    else:
        include_previous_pi_top_15 = pi_top_15

    # Export to Excel
    pi_top_15.to_excel('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'Most_Complex_Model_Permutation_Importance_Top_15.xlsx', index=False)

    # Export to Latex
    #pi_top_15.to_latex('../../../../Output/Modelling/Logistic Regression/Tables/Most_Complex_Model_Permutation_Importance_Top_15.tex', index=False)

    # Export to LaTeX
    # Center all columns
    lt_string = pi_top_15.to_latex(index=False, column_format='c' * len(pi_top_15.columns), escape=False)
    latex_with_font_size = "\\tiny\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/' + include_exclude_previous + 'Most_Complex_Model_Permutation_Importance_Top_15.tex', 'w') as f:
        f.write(latex_with_font_size)

    print(pi_top_15)

                                    Permuted Feature  Mean Accuracy Drop  \
0                                            Ratio E            0.070625   
1                                       Passive Tone            0.056786   
2                                  Sector: Utilities            0.043208   
3                                   Interest Expense            0.043019   
4                                            Ratio D            0.041765   
5                                            Ratio C            0.040578   
6   Depreciation and Amortization (Income Statement)            0.038593   
7                                    Net Receivables            0.036435   
8                                         Word Count            0.035743   
9                                     Long-Term Debt            0.035463   
10                             Market Capitalization            0.031103   
11                    Goodwill and Intangible Assets            0.030084   
12          

## Styling

In [11]:
include_previous_pi_top_15 = (include_previous_pi_top_15.reset_index(drop=True)
                            .style
                            .format(precision=6, thousands=",", decimal=".")
                            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                            .hide()
                            .set_properties(**{'text-align': 'center'}))

dfi.export(include_previous_pi_top_15, '../../../../Output/Modelling/' + classifier_name + '/Tables/' + 'include_previous_' + 'Most_Complex_Model_Permutation_Importance_Top_15.png')
#include_previous_pi_top_15

exclude_previous_pi_top_15 = (exclude_previous_pi_top_15.reset_index(drop=True)
                            .style
                            .format(precision=6, thousands=",", decimal=".")
                            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                            .hide()
                            .set_properties(**{'text-align': 'center'}))

dfi.export(exclude_previous_pi_top_15, '../../../../Output/Modelling/' + classifier_name + '/Tables/' + 'exclude_previous_' + 'Most_Complex_Model_Permutation_Importance_Top_15.png')
#exclude_previous_pi_top_15

## Change Model Comparison Tables

In [12]:
# Iterate over just models '_' versus _smote_
for spec in ['', 
             'smote_']:

    # Create list of df rows
    model_comparison_rows = []
    majority_baselines = []
    for mn, clean_model_name in zip(['rating_change_model_1', 'rating_change_model_2', 'rating_change_model_3'], ["Altman's Z", 'Financial Variables and Sector', 'Financial Variables, Sector, and NLP Features']):
        model_comparison_row, majority_baseline = get_model_comparison_row(spec + mn, clean_model_name)
        model_comparison_rows.append(model_comparison_row)
        majority_baselines.append(majority_baseline)

    # Concatenate rows
    model_comparison_df = pd.concat(model_comparison_rows)

    # Check majority baselines are the same
    print('Majority baselines are the same:', all([majority_baseline == majority_baselines[0] for majority_baseline in majority_baselines]))
    # Add row with Model/Baseline = 'Most Common Class Baseline' and Accuracy = majority_baseline[0]
    model_comparison_df = pd.concat([model_comparison_df, pd.DataFrame({
        'Model/Baseline': ['Most Common Class Baseline'],
        'Accuracy': [majority_baselines[0]],
        'Weighted Average Precision': [''],
        'Weighted Average Recall': [''],
        'F1 Score': [''],
        'Share 1 Rating Or Less From Actual': ['']
    })])
    # Drop 'Share 1 Rating Or Less From Actual' column
    model_comparison_df = model_comparison_df.drop(columns=['Share 1 Rating Or Less From Actual'])

    if spec == '':
        change_model_comparison_df = model_comparison_df
    else:
        change_smote_model_comparison_df = model_comparison_df

    # Export to Excel
    model_comparison_df.to_excel('../../../../Output/Modelling/' + classifier_name + '/Tables/change_' + spec + 'model_comparison_df.xlsx', index = False)

    # Export to LaTeX
    # Format columns
    for col in model_comparison_df.columns:
        model_comparison_df[col] = model_comparison_df[col].apply(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
    # Rename 'Share 1 Rating Or Less From Actual' to 'Share $le$ Rating From Actual'
    #model_comparison_df.rename(columns={'Share 1 Rating Or Less From Actual': 'Share $\\le$ 1 Rating From Actual'}, inplace=True)
    # Center all columns
    lt_string = model_comparison_df.to_latex(index=False, column_format='c' * len(model_comparison_df.columns), escape=False)
    latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/change_' + spec + 'model_comparison_df.tex', 'w') as f:
        f.write(latex_with_font_size)

    # Smaller version latex - keep only columns 'Model/Baseline', 'Accuracy'
    model_comparison_df_smaller = model_comparison_df[['Model/Baseline', 'Accuracy']]
    # Center all columns
    lt_string = model_comparison_df_smaller.to_latex(index=False, column_format='c' * len(model_comparison_df_smaller.columns), escape=False)
    latex_with_font_size = "\\footnotesize\n" + lt_string + "\n\\normalsize"
    with open('../../../../Output/Modelling/' + classifier_name + '/Tables/change_' + spec + 'model_comparison_df_smaller.tex', 'w') as f:
        f.write(latex_with_font_size)

    print(model_comparison_df)

exact predictions share == accuracy: True
exact predictions share == accuracy: True
exact predictions share == accuracy: True
Majority baselines are the same: True
                                  Model/Baseline Accuracy  \
0                                     Altman's Z   0.9535   
0                 Financial Variables and Sector   0.0286   
0  Financial Variables, Sector, and NLP Features   0.5984   
0                     Most Common Class Baseline   0.9535   

  Weighted Average Precision Weighted Average Recall F1 Score  
0                     0.9091                  0.9535   0.9308  
0                     0.0008                  0.0286   0.0016  
0                     0.9063                  0.5984   0.7189  
0                                                              
exact predictions share == accuracy: True
exact predictions share == accuracy: True
exact predictions share == accuracy: True
Majority baselines are the same: True
                                  Model/Baseli

In [13]:
# Styling
#print(exclude_previous_model_comparison_df)
change_model_comparison_df = (change_model_comparison_df.reset_index(drop=True)[['Model/Baseline', 'Accuracy']]
                            .style
                            .format(precision=2, thousands=",", decimal=".")
                            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                            .hide()
                            .set_properties(**{'text-align': 'center'}))

dfi.export(change_model_comparison_df, '../../../../Output/Modelling/' + classifier_name + '/Tables/' + 'change_model_comparison_df.png')
#change_model_comparison_df

change_smote_model_comparison_df = (change_smote_model_comparison_df.reset_index(drop=True)[['Model/Baseline', 'Accuracy']]
                            .style
                            .format(precision=2, thousands=",", decimal=".")
                            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                            .hide()
                            .set_properties(**{'text-align': 'center'}))

dfi.export(change_smote_model_comparison_df, '../../../../Output/Modelling/' + classifier_name + '/Tables/' + 'change_smote_model_comparison_df.png')
#change_smote_model_comparison_df